In [1]:
import numpy as np
import pandas as pd
import astroquery
import matplotlib.pyplot as plt
import glob

import matplotlib

%matplotlib inline

In [ ]:
fn = '/Users/tom/Dropbox/TIC5/ctl.csv'

header = [
    'RA','DEC',
    'ECLONG','ECLAT', 'Ks',
    'TESSMAG','TEFF','RADIUS','MASS','CONTRATIO','PRIORITY',
    ]
usecols = [13, 14, 60, 64, 87, 70, 72, 84, 26, 27, 46]
df = pd.read_csv(fn, names=header, usecols=usecols)

In [16]:
selectnum = int(np.round(400000/26., ))
df.loc[:,'SELECTED'] = np.zeros(df.shape[0], dtype='bool')

eloop = np.linspace(0,360,14)
espace = np.diff(eloop)[0] # get space in degrees
for elon in eloop[:-1]:
    # southern hemisphere
    ms = (df.ECLONG >= elon) & (df.ECLONG < elon+espace) & (df.ECLAT <= -6)
    selectthese = df.loc[ms].sort_values('PRIORITY').iloc[-selectnum:].index
    df.loc[df.index.isin(selectthese), 'SELECTED'] = True
    
#     # northern hemisphere
    mn = (df.ECLONG >= elon) & (df.ECLONG < elon+espace) & (df.ECLAT >= 6)
    selectthese = df.loc[mn].sort_values('PRIORITY').iloc[-selectnum:].index
    df.loc[df.index.isin(selectthese), 'SELECTED'] = True
    
#     print(df.loc[df.SELECTED == True].shape[0])
    print(elon,elon+espace,df.loc[df.SELECTED == True].shape[0])

0.0 27.6923076923 30770
27.6923076923 55.3846153846 61540
55.3846153846 83.0769230769 92310
83.0769230769 110.769230769 123080
110.769230769 138.461538462 153850
138.461538462 166.153846154 184620
166.153846154 193.846153846 215390
193.846153846 221.538461538 246160
221.538461538 249.230769231 276930
249.230769231 276.923076923 307700
276.923076923 304.615384615 338470
304.615384615 332.307692308 369240
332.307692308 360.0 400010


In [17]:
selected = df[df.SELECTED == True]

In [18]:
selected.to_csv('../data/selected5.csv.bz2',compression='bz2')

# ok let's now try to select things kinda realistically

In [ ]:
fn = '/Users/tom/Dropbox/TIC5/ctl.csv'

header = [
    'RA','DEC',
    'ECLONG','ECLAT', 'Ks',
    'TESSMAG','TEFF','RADIUS','MASS','CONTRATIO','PRIORITY',
    ]
usecols = [13, 14, 60, 64, 87, 70, 72, 84, 26, 27, 46]
df = pd.read_csv(fn, names=header, usecols=usecols)

In [ ]:
# have 7000 targets a each of the poles

selectnum = 8000
selectpoles = 6000
df.loc[:,'SELECTED'] = np.zeros(df.shape[0], dtype='bool')

eloop = np.linspace(0,360,14)
espace = np.diff(eloop)[0] # get space in degrees

#poles first
mn = (df.ECLAT >= 78)
selectthese = df.loc[mn].sort_values('PRIORITY').iloc[-selectpoles:].index
df.loc[df.index.isin(selectthese), 'SELECTED'] = True

mn = (df.ECLAT <= -78)
selectthese = df.loc[mn].sort_values('PRIORITY').iloc[-selectpoles:].index
df.loc[df.index.isin(selectthese), 'SELECTED'] = True

for elon in eloop[:-1]:
    # southern hemisphere
    ms = (df.ECLONG >= elon) & (df.ECLONG < elon+espace) & (df.ECLAT <= -6) & (df.ECLAT > -78)
    selectthese = df.loc[ms].sort_values('PRIORITY').iloc[-selectnum:].index
    df.loc[df.index.isin(selectthese), 'SELECTED'] = True
    
#     # northern hemisphere
    mn = (df.ECLONG >= elon) & (df.ECLONG < elon+espace) & (df.ECLAT >= 6) & (df.ECLAT < 78)
    selectthese = df.loc[mn].sort_values('PRIORITY').iloc[-selectnum:].index
    df.loc[df.index.isin(selectthese), 'SELECTED'] = True
    
    print(elon,elon+espace,df.loc[df.SELECTED == True].shape[0])

In [ ]:
selected = df[df.SELECTED == True]

In [6]:
selected.to_csv('../data/selectedreal5.csv.bz2',compression='bz2')

In [11]:
from astropy.coordinates import SkyCoord
from astropy import units as u

In [13]:
gc = SkyCoord(ra=150*u.degree, dec=45*u.degree, frame='icrs')

In [20]:
gc.barycentrictrueecliptic.lon.value

135.31169020376774